# Yearly cadence of an employee

The majority of the work force, specially office workers, follow the same rhythm of life. They have a weekend of 1 or 2 days, they take holidays off, and other than that they follow the schedule of their job. I probably didn't do a good job explaining this, but you get it. The machines don't get it though! In this notebook, I show an example of how to explain this to a machine by extracting features that will help it find such patterns in the data.

## Explainning the day of year to a machine

In statistics litrature, features are called explanatory variables. I like this name because it can be understood in two ways (and I like puns):

- The dry understand is that those variables explain the target or response variable; the LHS of the equation that explain the outcome on the RHS.
- The pun is that those variables explain some concepts to the machine, and by deciding to include them during feature engineering we are telling it to explore the importance of this concept.

Let's adopt the second understanding of explanatory variables, and let's see how to explain to the machine the concept of the yearly cadence of an employee. One might think of features such as `IsWeekend` which will be true if the day of week is Saturday or Sunday, and `IsThanksGiving` which will be true for American Thanks Giving day. This is OK if the user base is strictly in the USA. However, I think that there is too much of the programmer's biases encoded into these features. I prefer to engineer features that do not hard code my biases, but allow the machine to find the biases in the training data. This is another play on words, and not an exactly accurate use of the word bias as in Bias vs Variance -- however, I think it is no blatently wrong either, even though a more accurate word to use in this case is priors. Anyway, back to our example. Below is an enumeration of the explanatory variables I will use:

In [4]:
public enum DayOfYearExplanatoryVariables {
    IsMonday(0),
    IsTueday(1),
    IsWednesday(2),
    IsThursday(3),
    IsFriday(4),
    IsSaturday(5),
    IsSunday(6),
    IsJan(7),
    IsFeb(8),
    IsMar(9),
    IsApr(10),
    IsMay(11),
    IsJun(12),
    IsJul(13),
    IsAug(14),
    IsSep(15),
    IsOct(16),
    IsNov(17),
    IsDec(18),
    Year(19),
    IsHoliday(20),
    IsHolidayEve(21),
    IsWorkDay(22),
    IsDay01To10(23),
    IsDay11To20(24),
    IsDay21To31(25)
}

Name: Compile Error
Message: <console>:2: error: ';' expected but ',' found.
           IsMonday(0),
                      ^
StackTrace: 

## Dependencies

The code depends on Joda Time and Jolly Day:

- Joda Time is the de facto time library before Java 8. 
- Jolly Day is a library to detect holidays. It worked well for North American holidays, and it should work for other countries as well! Version 0.4.9 is the latest version that does not use Java 8 features, so I am using it to avoid dealing with issues of running Scala on Java 8. 

PS: Jolly Day pulls in a version of Joda Time.

In [5]:
%AddDeps de.jollyday jollyday 0.4.9 --transitive --verbose
import de.jollyday.{HolidayCalendar, HolidayManager}
import org.joda.time.format.ISODateTimeFormat
import org.joda.time.{DateTime, DateTimeConstants, DateTimeZone, Days}

Marking de.jollyday:jollyday:0.4.9 for download
Preparing to fetch from:
-> file:/var/folders/pd/3vnhbdwj3wx1nm58z5lnyyqm0000gn/T/toree_add_deps8827194317915505009/
-> https://repo1.maven.org/maven2
=> https://repo1.maven.org/maven2/de/jollyday/jollyday/0.4.9/jollyday-0.4.9.pom: Found at /var/folders/pd/3vnhbdwj3wx1nm58z5lnyyqm0000gn/T/toree_add_deps8827194317915505009/https/repo1.maven.org/maven2/de/jollyday/jollyday/0.4.9/jollyday-0.4.9.pom
=> https://repo1.maven.org/maven2/de/jollyday/jollyday/0.4.9/jollyday-0.4.9.pom.sha1: Found at /var/folders/pd/3vnhbdwj3wx1nm58z5lnyyqm0000gn/T/toree_add_deps8827194317915505009/https/repo1.maven.org/maven2/de/jollyday/jollyday/0.4.9/jollyday-0.4.9.pom.sha1
=> https://repo1.maven.org/maven2/de/jollyday/jollyday/0.4.9/jollyday-0.4.9.pom.sha1: Found at /var/folders/pd/3vnhbdwj3wx1nm58z5lnyyqm0000gn/T/toree_add_deps8827194317915505009/https/repo1.maven.org/maven2/de/jollyday/jollyday/0.4.9/jollyday-0.4.9.pom.sha1
=> https://repo1.maven.org/maven2/de/

Marking de.jollyday:jollyday:0.4.9 for download
Preparing to fetch from:
-> file:/var/folders/pd/3vnhbdwj3wx1nm58z5lnyyqm0000gn/T/toree_add_deps8827194317915505009/
-> https://repo1.maven.org/maven2
=> 1 (): Downloading https://repo1.maven.org/maven2/de/jollyday/jollyday/0.4.9/jollyday-0.4.9.pom.sha1
=> 2 (): Downloading https://repo1.maven.org/maven2/de/jollyday/jollyday/0.4.9/jollyday-0.4.9.pom
=> 1 (jollyday-0.4.9.pom.sha1): Finished downloading
=> 2 (jollyday-0.4.9.pom): Finished downloading
=> 3 (): Downloading https://repo1.maven.org/maven2/joda-time/joda-time/2.4/joda-time-2.4.pom.sha1
=> 4 (): Downloading https://repo1.maven.org/maven2/joda-time/joda-time/2.4/joda-time-2.4.pom
=> 5 (): Downloading https://repo1.maven.org/maven2/javax/xml/bind/jaxb-api/2.2.7/jaxb-api-2.2.7.pom.sha1
=> 6 (): Downloading https://repo1.maven.org/maven2/javax/xml/bind/jaxb-api/2.2.7/jaxb-api-2.2.7.pom
=> 3 (joda-time-2.4.pom.sha1): Finished downloading
=> 6 (jaxb-api-2.2.7.pom): Finished downloading